In [251]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


df = pd.read_csv('train.csv')

In [261]:
test_data = pd.read_csv('test5_all.csv')

In [240]:
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

In [241]:
#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

In [147]:
# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

In [176]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)

<216987x185595 sparse matrix of type '<class 'numpy.int64'>'
	with 1762366 stored elements in Compressed Sparse Row format>

In [177]:
print(len(vectorizer.vocabulary_))

185595


In [178]:
# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=['id'])

In [221]:
#LOAD DATA
train = pd.read_csv("corpus1.csv");

#CLEAN DATA
train['words'] = train['words'].str.replace("\d+", '')
train['words'] = train['words'].str.replace("sec", '')
train['words'] = train['words'].str.replace("nabkv", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
train['words'].replace('', np.nan, inplace=True)
train = train.dropna(subset=['words'])


In [229]:
train = train[train['words'].str.len() > 3]


In [230]:
# save data
train.to_csv("corpus.csv", encoding="utf-8")

In [281]:
# load data again 
test_data = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = test_data.loc[:,'words']

In [282]:
# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

In [288]:
vocab = tfidf.vocabulary_

In [337]:
# we have sparse matrix for 3 mln pictures 
# create KNN
train_data = pd.read_csv('dm_clean1.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

nbrs = NearestNeighbors(n_neighbors = 1).fit(train_c_matrix)

In [304]:
test_data = pd.read_csv('test5_all.csv')
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


In [338]:
for i in range(0,15):
    distances, indices = nbrs.kneighbors(test_c_matrix[i])

    ind = indices[0][0]
    print('ind',ind)
    print(distances)
    print(train_data.loc[ind,'latitude'])
    print(train_data.loc[ind,'longitude'])
    print('test:', test_data.loc[i,'photoTags'])
    print('train:', train_data.loc[ind,'photoTags'])
    print('-----------------------------')

ind 108549
[[ 0.]]
40.72
-73.99
test: fortune pei
train: nyc fortune
-----------------------------
ind 141351
[[ 0.80085474]]
47.52
-52.62
test: chimney rust stack cape spear
train: cape spear east07
-----------------------------
ind 1835208
[[ 0.45179511]]
47.57
-52.68
test: sunset car newfoundland lights stjohns signalhill cabottower
train: sign newfoundland stjohns signalhill cabottower fallhazard
-----------------------------
ind 276565
[[ 0.70794809]]
51.28
0.52
test: wifi broadband
train: broadband carphonewarehouse
-----------------------------
ind 25
[[ 1.]]
64.75
-152.28
test: geotagged coast australia cliffs sa southaustralia bundacliffs greataustralianbight geo:lat=31637306 geo:lon=129431133
train:  geolat647537192501493 geolon152277543501148
-----------------------------
ind 2463432
[[ 0.86529026]]
-31.65
128.97
test: geotagged sand dune wa westernaustralia eucla greataustralianbight geo:lat=3171634 geo:lon=128890057
train: australia westernaustralia eucla eremophila taxono